In [58]:
import json 
import uuid
from uuid import uuid4
# data manipulation
import pandas as pd 
import string
# normalize nested JSON files
from pandas.io.json import json_normalize
import os
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import PyPDF2, urllib.request , nltk , textract
from io import BytesIO
import json
#import weasyprint
import pandas as pd
from pandas.io.json import json_normalize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import os
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
from collections import defaultdict, Counter
from fuzzywuzzy import fuzz
import json
import logging
import multiprocessing as mp
import numpy as np
import pandas as pd
import os
import re
import regex
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('stopwords')  
nltk.download('punkt')

[nltk_data] Downloading package stopwords to C:\Users\Xiangyu
[nltk_data]     Ren\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Xiangyu
[nltk_data]     Ren\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [59]:
json_file_path = "OSU-Out-with-Thumbnails.json"
with open(json_file_path, 'r') as json_file: 
    data = json.load(json_file)

In [60]:
import ast
#grab the contents from OKU json
links = [x['link'] for x in data]
texts = [x['content'] for x in data]
displaydate = [x['displaydate'] for x in data]


<class 'list'>


In [73]:
df2 = pd.DataFrame(texts)
df2

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,39,40,41,42,43
0,"{'header': 'Introduction-w/o-header', 'text': ...","{'header': 'Hives in mind', 'text': 'Webster, ...","{'header': ''Amazing opportunity'', 'text': 'W...",None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,"{'header': 'Ask-Expert-Question', 'text': 'The...","{'header': 'Ask-Expert-Answer', 'text': 'It is...","{'header': 'Control tips for inside the home',...",{'header': 'Control tips for outside the home'...,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,"{'header': 'Ask-Expert-Question', 'text': 'I k...","{'header': 'Ask-Expert-Answer', 'text': 'Thank...",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,"{'header': 'Introduction-w/o-header', 'text': ...","{'header': 'Tests and costs', 'text': 'Samples...",{'header': 'Best practices for handling sample...,"{'header': 'Who to contact', 'text': 'If you h...",{'header': 'Where to ship or drop off samples ...,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,"{'header': 'Introduction-w/o-header', 'text': ...",{'header': 'Lawns help manage stormwater runof...,"{'header': 'Lawns help reduce urban heat.', 't...",{'header': 'But what about climate change? Are...,"{'header': 'So, where do lawns come in?', 'tex...",{'header': 'Tips for moving toward a more clim...,"{'header': 'Mindful mowing & management', 'tex...","{'header': 'Apply fertilizer thoughtfully', 't...","{'header': 'Use water wisely', 'text': 'If you...","{'header': 'Replace your water-hungry lawn', '...",...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,"{'header': 'Introduction-w/o-header', 'text': ...",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3138,"{'header': 'Introduction-w/o-header', 'text': ...",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3139,"{'header': 'Introduction-w/o-header', 'text': ...",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3140,"{'header': 'Introduction-w/o-header', 'text': ...",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [78]:
#make the data into dataframe
df1 = pd.DataFrame(links)
df1.columns = ['extension_links']
df2 = pd.DataFrame(texts)
df_new=df2.copy()
df_new['text'] = df_new[df_new.columns[0:43]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
)
#f_new['text'] = df_new[df_new.columns[0:110]].apply(
    #ambda x: ','.join(x.dropna().astype(str)),
    #axis=1
#)
df_new = df_new.drop(df_new.columns[0:44],axis=1)
df3 = pd.DataFrame(displaydate)
df3.columns = ['Displaydate']
all_data1= pd.concat([df1,df_new,df3],axis=1,join='inner')
all_data1['row_num'] = np.arange(len(all_data))


In [79]:
all_data1

,extension_links,text,Displaydate,row_num
0,https://extension.oregonstate.edu/news/master-...,"{'header': 'Introduction-w/o-header', 'text': ...",2022-10-06,0
1,https://extension.oregonstate.edu/ask-expert/f...,"{'header': 'Ask-Expert-Question', 'text': 'The...",2022-10-06,1
2,https://extension.oregonstate.edu/ask-expert/f...,"{'header': 'Ask-Expert-Question', 'text': 'I k...",2022-09-28,2
3,https://extension.oregonstate.edu/crop-product...,"{'header': 'Introduction-w/o-header', 'text': ...",2022-10-06,3
4,https://extension.oregonstate.edu/gardening/la...,"{'header': 'Introduction-w/o-header', 'text': ...",2022-10-05,4
...,...,...,...,...
3137,https://extension.oregonstate.edu/news/how-lon...,"{'header': 'Introduction-w/o-header', 'text': ...",2020-05-18,3137
3138,https://extension.oregonstate.edu/news/how-con...,"{'header': 'Introduction-w/o-header', 'text': ...",2020-05-18,3138
3139,https://extension.oregonstate.edu/news/what-mu...,"{'header': 'Introduction-w/o-header', 'text': ...",2020-05-18,3139
3140,https://extension.oregonstate.edu/news/ladybug...,"{'header': 'Introduction-w/o-header', 'text': ...",2020-05-18,3140


In [80]:
#create unique id based on date,OKU, and the unique number of document
all_data1['unique_id'] = all_data1['Displaydate'].astype(str)+'-'+all_data1['row_num'].astype(str)+'-'+'OSU'
all_data1

,extension_links,text,Displaydate,row_num,unique_id
0,https://extension.oregonstate.edu/news/master-...,"{'header': 'Introduction-w/o-header', 'text': ...",2022-10-06,0,2022-10-06-0-OSU
1,https://extension.oregonstate.edu/ask-expert/f...,"{'header': 'Ask-Expert-Question', 'text': 'The...",2022-10-06,1,2022-10-06-1-OSU
2,https://extension.oregonstate.edu/ask-expert/f...,"{'header': 'Ask-Expert-Question', 'text': 'I k...",2022-09-28,2,2022-09-28-2-OSU
3,https://extension.oregonstate.edu/crop-product...,"{'header': 'Introduction-w/o-header', 'text': ...",2022-10-06,3,2022-10-06-3-OSU
4,https://extension.oregonstate.edu/gardening/la...,"{'header': 'Introduction-w/o-header', 'text': ...",2022-10-05,4,2022-10-05-4-OSU
...,...,...,...,...,...
3137,https://extension.oregonstate.edu/news/how-lon...,"{'header': 'Introduction-w/o-header', 'text': ...",2020-05-18,3137,2020-05-18-3137-OSU
3138,https://extension.oregonstate.edu/news/how-con...,"{'header': 'Introduction-w/o-header', 'text': ...",2020-05-18,3138,2020-05-18-3138-OSU
3139,https://extension.oregonstate.edu/news/what-mu...,"{'header': 'Introduction-w/o-header', 'text': ...",2020-05-18,3139,2020-05-18-3139-OSU
3140,https://extension.oregonstate.edu/news/ladybug...,"{'header': 'Introduction-w/o-header', 'text': ...",2020-05-18,3140,2020-05-18-3140-OSU


In [6]:
#save it to csv format
all_data.to_json("all_data.json")


In [82]:
all_data1.set_index('unique_id', inplace=True)
all_data1

,extension_links,text,Displaydate,row_num
unique_id,,,,
2022-10-06-0-OSU,https://extension.oregonstate.edu/news/master-...,"{'header': 'Introduction-w/o-header', 'text': ...",2022-10-06,0
2022-10-06-1-OSU,https://extension.oregonstate.edu/ask-expert/f...,"{'header': 'Ask-Expert-Question', 'text': 'The...",2022-10-06,1
2022-09-28-2-OSU,https://extension.oregonstate.edu/ask-expert/f...,"{'header': 'Ask-Expert-Question', 'text': 'I k...",2022-09-28,2
2022-10-06-3-OSU,https://extension.oregonstate.edu/crop-product...,"{'header': 'Introduction-w/o-header', 'text': ...",2022-10-06,3
2022-10-05-4-OSU,https://extension.oregonstate.edu/gardening/la...,"{'header': 'Introduction-w/o-header', 'text': ...",2022-10-05,4
...,...,...,...,...
2020-05-18-3137-OSU,https://extension.oregonstate.edu/news/how-lon...,"{'header': 'Introduction-w/o-header', 'text': ...",2020-05-18,3137
2020-05-18-3138-OSU,https://extension.oregonstate.edu/news/how-con...,"{'header': 'Introduction-w/o-header', 'text': ...",2020-05-18,3138
2020-05-18-3139-OSU,https://extension.oregonstate.edu/news/what-mu...,"{'header': 'Introduction-w/o-header', 'text': ...",2020-05-18,3139


In [84]:
for i in all_data1.index:
    all_data1.loc[i].to_json("{}.json".format(i))